## Main configs

In [1]:
#only run one time to install all the needed packages for the project

# !pip install bs4
# !pip install matplotlib
# !pip install numpy
# !pip install requests
# !pip install httplib2
# !pip install nltk
# !pip install import_ipynb
# !pip install textacy
# !pip install spacy
# !pip install scispacy
# !pip install openpyxl
!pip install joblib

# !python -m spacy download en_core_web_lg

# import nltk
# nltk.download('punkt')
# nltk.download('treebank')
# nltk.download('omw-1.4')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_treebank_pos_tagger')
# nltk.download('stopwords')
# nltk.download('wordnet')

You should consider upgrading via the 'c:\users\makm7\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
from functions import *

t = time.time()
start = time.time()
abbrv = get_data(abbreviation_location)
nlp = spacy.load("en_core_web_lg")

write('loading time: ' + str(time.time() - t))
t = time.time() ##

for category in categories:
    log.write('\n' + category + '\n')

    path = result_path + '/combined/' + category + '.txt'

    txt = open(path, 'r', encoding='utf-8').read().strip()
    sentences = sent_tokenize(txt)

    tokenized = [word_tokenize(sentence) for sentence in sentences]
    posTags = [nltk.pos_tag(token) for token in tokenized]

    docs = [nlp(sentence) for sentence in sentences]
    txtLength = len(txt.split())
    write( category + ' with text length ' + str(txtLength/1000) + 'k words took: ' + str(time.time() - t) + ' to process ')

    t = time.time() ###

    for rule in rules_results.keys():
        if rule == 'Abbreviation':
            rule_occ = rules_fun[rule] (txt.lower(), abbrv)
        else:
            rule_occ = sum( rules_fun[rule] (doc, posTag)  for (doc,posTag) in zip(docs,posTags))

        rules_results[rule][category] = 100 * rule_occ / txtLength

        write( str(category) + '\t' + str( rules_abbrv[rule]) + '\t' + str(time.time() - t) + '\tocc: ' +  str(rules_results[rule][category])) ###
        t = time.time() ###
    write ('\n')

write('\n\nTotal time: ' + str(time.time() - start))


loading time: 10.689108848571777
tech with text length 830.397k words took: 1161.158462524414 to process 
tech	CC	3.128934383392334	occ: 7.321076545315072
tech	PV	2.4568755626678467	occ: 5.587327507204385
tech	AB	15.3589608669281	occ: 10.193437596715787
tech	SC	0.2799248695373535	occ: 10.625881355544395
tech	PPN	0.1460247039794922	occ: 1.2304957749124816


sport with text length 729.003k words took: 859.646942615509 to process 
sport	CC	0.7172844409942627	occ: 5.5600594236237715
sport	PV	0.8350818157196045	occ: 6.380906525761897
sport	AB	11.595850706100464	occ: 8.259499617971393
sport	SC	0.1678006649017334	occ: 10.174580900215775
sport	PPN	0.13921260833740234	occ: 1.8015015027373


business with text length 896.186k words took: 1043.6737976074219 to process 
business	CC	0.9413409233093262	occ: 6.08645973045774
business	PV	0.9735069274902344	occ: 5.807053446494366
business	AB	15.14260983467102	occ: 11.673581153912245
business	SC	0.20058560371398926	occ: 10.770978346012992
business	PPN	0

KeyboardInterrupt: 

In [ ]:

for rule in rules_results.keys():
    save(result_path + '/word_statistics/' + rule + '.csv', rules_results[rule])
for rule in rules_results.keys():
    graph(rule, rules_results[rule])
    plt.savefig(result_path+ '/word_statistics/'  + rule + '.eps', format='eps')
    plt.savefig(result_path+ '/word_statistics/'  + rule + '.png', format='png')
    plt.close()

In [5]:
from functions import *
save('Coordinary Conjunction.csv', rules_results['Coordinary Conjunction'])

In [1]:
from joblib import Parallel, delayed
from joblib import parallel_backend

import spacy
nlp = spacy.load("en_core_web_lg")

def chunker(iterable, total_length, chunksize):
    print('chunking texts')
    return [iterable[pos: pos + chunksize] for pos in range(0, total_length, chunksize)]

def flatten(list_of_lists):
    "Flatten a list of lists to a combined list"
    return [item for sublist in list_of_lists for item in sublist]

def process_chunk(texts):
    print('prepocessing chunk')
    preproc_pipe = []
    for doc in nlp.pipe(texts, batch_size=20):
        preproc_pipe.append(doc)
    return preproc_pipe

def preprocess_parallel(texts, n_threads = 2 ,chunksize=100):
    print('prepocessing texts')
    print('1')
    do = delayed(process_chunk)
    print('2')
    tasks = [do(chunk) for chunk in chunker(texts, len(texts), chunksize=chunksize)]
    print('3')
    with parallel_backend('threading', n_jobs=n_threads):
        result = Parallel()(tasks)
    return flatten(result)

In [2]:
from mainConfig import *
path = result_path + '/bbc/texts/entertainment/' 
txt = [open(path + fileName, 'r', encoding='utf-8').read().strip() for fileName in os.listdir(path) if fileName.endswith('.txt')]

In [3]:
res = preprocess_parallel(txt, chunksize=50)

prepocessing texts
1
2
chunking texts
3
prepocessing chunk
prepocessing chunk
prepocessing chunk
prepocessing chunk
prepocessing chunk
prepocessing chunk
prepocessing chunk
prepocessing chunk
prepocessing chunk


In [3]:
len(txt)

412

In [9]:
chs = chunker(txt, len(txt), chunksize = 50)

chunking texts


In [10]:
len(chs)

9

In [12]:

do = delayed(process_chunk)

In [16]:

tasks = [do(chunk) for chunk in chs]

In [ ]:
tasks[0]

In [22]:
len(flatten(result))

412

In [5]:
tasks = preprocess_parallel(txt, chunksize=20)

prepocessing texts
1
2
chunking texts


In [7]:
executor = Parallel(n_jobs=4, backend='multiprocessing', prefer="processes")
executor(tasks[0])

TypeError: 'generator' object is not subscriptable